# Resetting the Manager

<div class="alert alert-block alert-info">
<i>Hello there.!<br></i> Welcome to <b>python-iLOrest-library</b> <br> Here is an example to <b>Resetting the Manager</b> <br> As you are here let's get it done easily.<br>
    To perform this u need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style = "none">python-ilorest-library</a> installed in your machine.<br>
    And please keep your username and password handy!
    <br>
    <b>Also do execute each successive cell once to avoid any errors!</b>
    
</div>

<div class="alert alert-block alert-success">
    <h1> 1.</h1>
Below we are importing all the necessary modules from the ilorest library!.<br>
    <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys </a>
    </li>
    <li>json </li>
    <li> redfish </li>
    <li>ServerDownOrUnreachableError</li>
    <li>get_resource_directory</li>
        
</div>

In [1]:
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError
from get_resource_directory import get_resource_directory

<div class = "alert alert-block alert-success">
    <h1> 2. </h1>This is our clear_ahs_data to function overview, in the end this will clear ahs data.<br>
    <i>Let me give you a walk through of it!</i>
    <br>
 There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to change the settings.<br>

1. In the <b>resetting a manager</b> function we are first using <b>resource_instances</b> variable to obtain information about the resource directory by using get_resource_directory module.
<br>
    
2. If the resource instance is not empty then the code fetches <b>information about a manager</b> from a collection of "Managers" resources on a target system, using a Redfish client object. It retrieves the first member's URI, and then gets more details about that specific member.
    <br>

3. Else i.e; then iterates through a collection of Redfish resource instances and checks if the type of each instance is <b>#Manager</b>. If it is, the code retrieves more information about that specific instance using the Redfish client object <b>_redfishobj</b>.
    <br>

4. If manager_members_respone is ture then it retrieves the URI for the <b>Reset</b> action on a specific Redfish manager using the <b>managers_members_response</b> variable, and then creates a request body containing the <b>Action</b> key with the value <b>Manager.Reset</b>. Finally, it sends a <i>POST</i>request to the retrieved URI using the Redfish client object <b>_redfishobj</b> to initiate a reset operation on the target system.
    <br>
 5.Next is to check the status of the response received from the previous <b>POST</b> request sent using the Redfish client object <b>_redfishobj</b>.
    <br>
        1.If the status is 400, it attempts to print the extended error message associated with the response.
    <br>
        2.If the response status is not 400 and not 200, an error message is printed to the standard error stream.
    <br>
        3.If the response status is 200, it indicates that the reset operation was successful. The code then prints           a success message and prints the JSON response received from the server in a formatted way.
    <br>

 </div>

In [1]:
def reset_ilo(_redfishobj, DISABLE_RESOURCE_DIR):

    managers_members_response = None
    
    #function get_resource_directory returns Redfish resource instances from client object.
    resource_instances = get_resource_directory(_redfishobj)
    #checking target resource is empty are not 
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        managers_uri = _redfishobj.root.obj['Managers']['@odata.id']
        managers_response = _redfishobj.get(managers_uri)
        managers_members_uri = next(iter(managers_response.obj['Members']))['@odata.id']
        managers_members_response = _redfishobj.get(managers_members_uri)
    else:
        #Use Resource directory to find the relevant URI
        for instance in resource_instances:
            if '#Manager.' in instance['@odata.type']:
                managers_members_uri = instance['@odata.id']
                managers_members_response = _redfishobj.get(managers_members_uri)
   #checking the managers_members_response true or not
    if managers_members_response:
        reset_ilo_uri = managers_members_response.obj['Actions']['#Manager.Reset']['target']
        body = {'Action': 'Manager.Reset'}
        resp = _redfishobj.post(reset_ilo_uri, body)

    #If iLO responds with soemthing outside of 200 or 201 then lets check the iLO extended info
    #error message to see what went wrong
    if resp.status == 400:
        try:
            print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, sort_keys=True))
        except Exception as excp:
            sys.stderr.write("A response error occurred, unable to access iLO Extended Message "\
                             "Info...")
    elif resp.status != 200:
        sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
    else:
        print("Success!\n")
        print(json.dumps(resp.dict, indent=4, sort_keys=True))


<div class = "alert alert-block alert-success">
    <h1> 3. </h1>This is Main function overview.<br>
    <i>Let me give you a walk through of it!</i>
    <br>
 There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to change the settings.<br>

1. The rest opertions are handled by redfish object which is created by RedfishClient<br>
<br>
    
2. <b>RedfishClient</b> class takes 3 parameters<i> iLO url, Username,Password</i> and returns a redfish object.<br>
    <br>

3. Once you enter everything a redfish obejct is created with the help of RedfishClient class.<br>
    <br>

4. ow we try to Login with the help of <i> REDFISHOBJ</i>, if it is successful a session is created and you are logged in, But if           Server       is not avaialbe we get an error saying "server unreachable"<br>
    <br>
5. Then we call <b>reset_ilo</b>function with two parameters <i> redfish object,DISABLE_RESOURCE_DIR</i>  with our redfish object and rest the settings to default.
    <br>
    

6. REDFISHOBJ.logout() will take care of logging you out of the ilo.

 </div>

In [ ]:
if __name__ == "__main__":
    # When running on the server locally use the following commented values
    #SYSTEM_URL = None
    #LOGIN_ACCOUNT = None
    #LOGIN_PASSWORD = None

    # When running remotely connect using the secured (https://) address,
    # account name, and password to send https requests
    # SYSTEM_URL acceptable examples:
    # "https://10.0.0.100"
    # "https://ilo.hostname"
    SYSTEM_URL = "https://10.0.0.100"
    LOGIN_ACCOUNT = "admin"
    LOGIN_PASSWORD = "password"
     # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = True

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    reset_ilo(REDFISHOBJ, DISABLE_RESOURCE_DIR)
    REDFISHOBJ.logout()

<div class="alert alert-block alert-warning">
<b> When running on the server locally : </b>  use the following values <br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br>
    <b> When running remotely </b> connect using the secured (https://) address,account name, and password to send https requests <br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>

<div class = "alert alert-block alert-success">In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library:<a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.Hpe ilorest-api explorer:<a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.Rest api: <a href = "https://restfulapi.net/">LINK</a><br>
    
</div>